In [27]:
import win32api, win32con, win32gui, win32ui
from PIL import ImageGrab
import time
from config import *
import numpy as np
from utils import Color, pic_shot

In [40]:
class Cursor(object):
    def __init__(self, init_pos):
        # init_pos：should be a tuple, set `False` to skip initing position.
        if init_pos!=False and len(init_pos)==2:
            self.move_to(init_pos)
    
    def move_to(self, pos):
        win32api.SetCursorPos(pos)
    
    def get_pos(self):
        return win32api.GetCursorPos()
    
    def click(self):
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0, 0, 0)
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0, 0, 0)
    
    def right_click(self):
        win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTDOWN, 0, 0, 0, 0) 
        win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTUP, 0, 0, 0, 0)

In [28]:
class Fgo(object):
    def __init__(self, full_screen=True):
        # [init by yourself] put cursor at the down-right position of the game window.
        self.coloer = Color()
        
        if full_screen:
            self.height = win32api.GetSystemMetrics(win32con.SM_CYSCREEN)
            self.width = win32api.GetSystemMetrics(win32con.SM_CXSCREEN)
            self.scr_pos1 = (0, 0)
            self.scr_pos2 = (self.width, self.height)
            self.c = Cursor(init_pos=False)
            for x in range(5):
                print('[INFO {}] Program will start in {} s, Please enter FULL SCREEN MODE.'.format(
                    time.strftime('%H:%M:%S'), x+1))
            
        else:
            self.c = Cursor(init_pos=False)
            while 1:
                input('[INFO {}] Move cursor to the top-left cornor of Fgo screen, and press ENTER:'.format(
                    time.strftime('%H:%M:%S')))
                self.scr_pos1 = self.c.get_pos()
                print('[INFO {}] Get cursor at {}'.format(time.strftime('%H:%M:%S'), self.scr_pos1))
                
                input('[INFO {}] Move cursor to the down-right cornor of Fgo screen, and press ENTER:'.format(
                    time.strftime('%H:%M:%S')))
                self.scr_pos2 = self.c.get_pos()
                print('[INFO {}] Get cursor at {}'.format(time.strftime('%H:%M:%S'), self.scr_pos2))
                
                res = input('[INFO {}] Continue?[y/n](y to continue, n to reset cursor):'.format(
                    time.strftime('%H:%M:%S')))
                if res == 'n':
                    continue
                else: 
                    break
            for x in range(5):
                print('[INFO {}] Program will start in {} s, Please make sure the game window not covered.'.format(
                time.strftime('%H:%M:%S'), x+1))        

            self.width = abs(self.scr_pos2[0] - self.scr_pos1[0])
            self.height = abs(self.scr_pos2[1] - self.scr_pos1[1])
        
        
        #---------------------sampled pix info-----------------------
        # postion of samples:
        self.sample1_x = 0.8854
        self.sample1_y = 0.7824
        self.sample1_RGB = np.array([0, 0, 0])
        
        print('\n---------------------[init over]---------------------------')
        print('[DEBUG {}] Window width(x) = {}, height(y) = {}'.format(
            time.strftime('%H:%M:%S'), self.width, self.height))
        
    
    def _set(self, float_x, float_y):
        # input type: float
        return int(self.scr_pos1[0]+self.width*float_x), int(self.scr_pos1[1]+self.height*float_y)
    
    def click_act(self, float_x, float_y, sleep_time, click=True):
        pos = self._set(float_x, float_y)
        self.c.move_to(pos)
        if click:
            self.c.click()
        print('[INFO {}] simulate click at position: {}'.format(time.strftime('%H:%M:%S'), pos))
        time.sleep(sleep_time)
    
    def enter_battle(self, supNo=8):
        # [init by yourself] put the tag of battle at the top of screen.
        
        # postion of the center of battle tag.
        bat_tag_y = 0.2740
        bat_tag_x = 0.7252
        self.click_act(bat_tag_x, bat_tag_y, 1)
        
        # choose support:
        # postion of support servant tag.
        sup_tag_x = 0.4893
        sup_tag_y = 0.3944
        # postion of support class icon.
        sup_ico_x = 0.0729+0.0527*supNo
        sup_ico_y = 0.1796
        
        self.click_act(sup_ico_x, sup_ico_y, 0.5)
        self.click_act(sup_tag_x, sup_tag_y, 1.3)
        
        # game start
        # postion of `mission start` tag
        start_x = 0.9281
        start_y = 0.9398
        self.click_act(start_x, start_y, 1)
        
    def compare_RGB(self, target, x, y):
        '''
        - x, y: position of pix in Fgo window .
        - target: a numpy array of target RGB value.
        '''
        size = target.shape()
        
    def use_skill(self, skills):
        # position of skills:
        ski_x = [0.0542, 0.1276, 0.2010, 0.3021, 0.3745, 0.4469, 0.5521, 0.6234, 0.6958]
        ski_y = 0.8009
        # snap = 0.0734
        for i in skills:
            # SKILL_SLEEP_TIME = 0.8
            self.click_act(ski_x[i], ski_y, SKILL_SLEEP_TIME)
              
        
    def attack():
        # attack icon position:
        atk_ico_x = 0.8823
        atk_ico_y = 0.8444
        self.click_act(atk_ico_x, atk_ico_y, 0.8)
        
        # use atk card:
        # use ultimate atk card:
        if USE_ULTIMATE:
            # ultimate card position:
            ult_x = [0.3171, 0.5005, 0.6839]
            ult_y = 0.2833
            for x in ult_x:
                self.click_act(x, ult_y, ULTIMATE_SLEEP_TIME)
        
        # use normal atk card:
        # normal atk card position:
        atk_card_x = [0.1003+0.2007*x for x in range(5)]
        atk_card_y = [0.7019]
        for i in range(3):
            self.click_act(atk_card_x[i], atk_card_y, 0.3)
            
            
            
        
        
        

In [33]:
x = 192.5
x = 
y = 758
x/1920, y/1080


(0.10026041666666667, 0.7018518518518518)

In [47]:
c = Cursor(init_pos=False)
for x in range(5):
    c.move_to((0, x*20))
    time.sleep(0.5)

In [23]:
fgo = Fgo(full_screen=FULL_SCREEN)
# fgo.enter_battle(SUPPORT)
# fgo.use_skill(USED_SKILL)
fgo.attack()

print('done!')

[INFO 14:11:17] Move cursor to the top-left cornor of Fgo screen, and press ENTER:
[INFO 14:11:21] Get cursor at (23, 178)
[INFO 14:11:21] Move cursor to the down-right cornor of Fgo screen, and press ENTER:
[INFO 14:11:24] Get cursor at (787, 605)
[INFO 14:11:24] Continue?[y/n](y to continue, n to reset cursor):

---------------------[init over]---------------------------
[DEBUG 14:11:26] Window width(x) = 764, height(y) = 427
[INFO 14:11:26] simulate click at position: (577, 294)
[INFO 14:11:27] simulate click at position: (360, 254)
[INFO 14:11:28] simulate click at position: (396, 346)
[INFO 14:11:29] simulate click at position: (732, 579)
[INFO 14:11:32] simulate click at position: (64, 519)
[INFO 14:11:33] simulate click at position: (120, 519)
[INFO 14:11:34] simulate click at position: (176, 519)
[INFO 14:11:35] simulate click at position: (253, 519)
[INFO 14:11:36] simulate click at position: (309, 519)
[INFO 14:11:36] simulate click at position: (364, 519)
[INFO 14:11:37] sim

In [ ]:
'''w = win32api.GetSystemMetrics(win32con.SM_CXSCREEN)
h = win32api.GetSystemMetrics(win32con.SM_CYSCREEN)
print(w, h)
c = Cursor(init_pos=False)
c.get_pos()
# c = Cursor(init_pos=False)
# c.right_click()
#c.move_to((w, h))'''